# T5 on TPU 💥🚀

In [ ]:
VERSION = "20211010"  #@param ["1.5" , "20211010", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  28903      0 --:--:-- --:--:-- --:--:-- 28903
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20211010 ...
Found existing installation: torch 1.10.0+cu111
     |████████████████████████████████| 57 kB 2.6 MB/s 
Uninstalling torch-1.10.0+cu111:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.290 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompati

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.4 MB/s 


Install transformers and the nlp package. Restart colab after this

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install -U nlp

Cloning into 'transformers'...
remote: Enumerating objects: 95191, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 95191 (delta 0), reused 5 (delta 0), pack-reused 95185
Receiving objects: 100% (95191/95191), 80.43 MiB | 24.37 MiB/s, done.
Resolving deltas: 100% (69106/69106), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61 kB 441 kB/s 
     |████████████████████████████████| 596 kB 11

## Load and process data

In [ ]:
import os 
os.environ['LD_LIBRARY_PATH']='/usr/local/lib'

!echo $LD_LIBRARY_PATH
!sudo ln -s /usr/local/lib/libmkl_intel_lp64.so /usr/local/lib/libmkl_intel_lp64.so.1
!sudo ln -s /usr/local/lib/libmkl_intel_thread.so /usr/local/lib/libmkl_intel_thread.so.1
!sudo ln -s /usr/local/lib/libmkl_core.so /usr/local/lib/libmkl_core.so.1

!ldconfig
!ldd /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch.so

/usr/local/lib
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

	linux-vdso.so.1 (0x00007ffd68985000)
	/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4 (0x00007f0323e06000)
	libtorch_cpu.so => /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch_cpu.so (0x00007f03194c0000)
	libc10.so => /usr/local/lib/python3.7/dist-packages/torch/lib/libc10.so (0x00007f0319262000)
	libmkl_intel_lp64.so.1 => /usr/local/lib/libmkl_intel_lp64.so.1 (0x00007f0318731000)
	libmkl_intel_thread.so.1 => /usr/local/lib/libmkl_intel_thread.so.1 (0x00007f0316251000)
	libmkl_core.so.1 => /usr/local/lib/libmkl_core.so.1 (0x00007f0312118000)
	libomp.so.5 => /usr/lib/x86_64-linux-gnu/libomp.so.5 (0x00007f0311e63000)
	libpthread.so.0 => /lib/x86_64-linux-gnu/libpthread.so.0 (0x00007f0311c44000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007f03118a6000)
	libdl.so.2 => /lib/x86_64-linux-gnu/libdl.so.2 (0x00007f03116a2000)
	libstdc++.so.6 => /usr/lib/x8

In [ ]:
import torch
import nlp
from transformers import T5Tokenizer

In [ ]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question_translated'], example['context_translated'])
    example['target_text'] = '%s </s>' % example['text_translated']
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=1024)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=32)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }
    

    return encodings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Evaluation on SLQuAD dataste

In [ ]:
## SQuAD evaluation script. Modifed slightly for this Sinhala QA task

from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    # prediction_tokens = normalize_answer(prediction).split()
    # ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction) & Counter(ground_truth)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction)
    recall = 1.0 * num_same / len(ground_truth)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (prediction == ground_truth)


def metric_max_over_ground_truths(metric_fn, prediction, ground_truth):
    scores_for_ground_truths = []
    score = metric_fn(prediction, ground_truth)
    scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truths, prediction in zip(gold_answers, predictions):
      total += 1
      exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
      f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

import nlp
from transformers import T5ForConditionalGeneration, T5Tokenizer

from tqdm.auto import tqdm

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('sankhajay/mt5-base-sinaha-qa').to('cpu') # because its loaded on xla by default
tokenizer = T5Tokenizer.from_pretrained('sankhajay/mt5-base-sinaha-qa')
# tokenizer = T5Tokenizer.from_pretrained('models/tpu')

Downloading:   0%|          | 0.00/680 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/_utils.py:134: UserWarning: Failed to initialize NumPy: module compiled against API version 0xe but this version of numpy is 0xd (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:68.)
  t = torch.tensor([], dtype=storage.dtype, device=storage._untyped().device)


Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
eval_dataset = pd.read_csv('/content/drive/My Drive/4th Year/4th year research/dataset/si-wiki-qa-original-cleaned-new.csv')

In [ ]:
eval_dataset.head()

,Unnamed: 0,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,1,ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් ...,ශ්රී දළදා මාළිගාව පිහිටා ඇති නගරය ලෝක උරුමයක් ...,යුනෙස්කෝව,604,614
1,1,3,ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් ...,දැනට දක්නට ලැබෙන පැරණි දෙමහල් වැඩසිටින මාලිගාව...,ශ්රී වීරපරාක්රම නරේන්ද්රසිංහ,436,464
2,2,5,තාරකා විද්යාව හෙවත් තාරකාවේදය යනු තාරකා ග්රහලෝ...,තාරකා විද්යාවෙ නිරීක්ෂණයන් වැදගත් වන්නේ මොනවාටද,සාමාන්ය සාපේක්ෂතා වාදය වැනි භෞතික විද්යාත්මක ම...,397,464
3,3,6,තාරකා විද්යාව හෙවත් තාරකාවේදය යනු තාරකා ග්රහලෝ...,නූතන තාරකා විද්යාවට සමාන ක්ෂේත්රයන් මොනවාද,ජ්යෝතීෂ්යය,956,966
4,4,7,තාරකා විද්යාව හෙවත් තාරකාවේදය යනු තාරකා ග්රහලෝ...,තාරකා විද්යාවේ අනුගමනය කරන ක්රමවේදය කුමක්ද,විද්යාත්මක ක්රමය,512,529


In [ ]:
eval_dataset.iloc[200]['context_translated']

'සෘජු තෝරාගැනීම යනු දේශපාලන තනතුරුධාරීන් තෝරාගැනීමේ ක්රමයක් විස්තර කෙරෙන පදයක් වන අතර තේරී පත්වීම සඳහා අභිමත පුද්ගලයා පුද්ගලයන් හෝ දේශපාලන පක්ෂය සඳහා ඡන්දදායකයන් විසින් මෙහිදී සෘජුව ඡන්දය ප්රකාශ කරනු ලැබේ සෘජු මැතිවරණයකදී ජයග්රාහකයා හෝ ජයග්රාහකයන් හෝ තෝරාගන්නා ක්රමය පදනම් වන්නේ භාවිතා වන මැතිවරණ ක්රමය මතය වඩාත් බහුල ලෙසින් භාවිතා වන ක්රම වන්නේ බහුත්ව ක්රමය සහ ජනාධිපතිවරණ වැනි තනි ජයග්රාහක මැතිවරණ සඳහා වට දෙකෙහි ක්රමය සහ ව්යවස්ථාදායක මැතිවරණයක් සඳහා පක්ෂලැයිස්තු සමානුපාතික නියෝජන ක්රමය යන්නන් වෙති සෘජු ලෙසින් තේරී පත්වන මණ්ඩල සඳහා නිදසුන් වන්නේ යුරෝපියානු පාර්ලිමේන්තුව 1979 වසරෙහි සිට සහ එක්සත් ජනපද නියෝජිත මන්ත්රී මණ්ඩලය 1788 වසරෙහි සිට1 මෙයට හාත්පසින්ම විරුද්ධ ලෙසින් වක්රාකාර තෝරාගැනීමක් තුලදී ඡන්දදායකයන් විසින් මණ්ඩලයක් තෝරාගනු ලබන අතර එය විසින් ඉන්පසුව ප්රශ්නයට තුඩුදී ඇති තනතුරුධාරියා තෝරා පත් කර ගනියි මෙයට නිදසුනක් වන්නේ එජ මැතිවරණ පර්ෂදය වන අතර ඡන්දදායකයන් විසින් තෝරා පත්කරනු ලබන මෙහි අභිනියෝජිතයන් විසින් ඉන්පසුව ඊළඟ ජනාධිපතිවරයා තෝරා පත්කරනු ලබයි'

In [ ]:
import json
columns = ['question_translated','text_translated','context_translated','answer_start','answer_end']

In [ ]:
!pip install datasets

     |████████████████████████████████| 306 kB 5.4 MB/s 
     |████████████████████████████████| 132 kB 53.8 MB/s 
     |████████████████████████████████| 1.1 MB 38.6 MB/s 
     |████████████████████████████████| 192 kB 61.9 MB/s 
     |████████████████████████████████| 160 kB 55.9 MB/s 
     |████████████████████████████████| 271 kB 75.9 MB/s 


In [ ]:
from datasets import Dataset

eval = Dataset.from_pandas(eval_dataset.loc[0:,columns])

In [ ]:
eval[1]

{'answer_end': 464,
 'answer_start': 436,
 'context_translated': 'ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් දන්තධාතූන් වහන්සේ වර්තමානයේ තැන්පත් කර ඇති දළදා මාළිගාවයි වර්තමාන දළදා මාළිගාව ශ්රී ලංකාවේ මහනුවර නගරයේ පිහිටා ඇත උඩරට රාජ්යය සමයේ මෙය පිහිටා තිබුණේ එවකට පැවති රාජකීය මාලිගා සංකීර්ණය තුළමය ශ්රී දළදා මාලිගාව මුල්වරට ගොඩනගනු ලැබුයේ පළමුවන විමලධර්මසූරිය රජතුමා විසිනි එතුමා ඉදිකළ ශ්රී දළදා මාලිගාව පෘතුගීසි ආක්රමණිකයෝ විනාශ කළහ දැනට දක්නට ලැබෙන පැරණි දෙමහල් වැඩසිටින මාලිගාව ඉදිකරනු ලැබ ඇත්තේ ශ්රී වීරපරාක්රම නරේන්ද්රසිංහ රජතුමාය ඉන්පසු රජ පැමිණි නායක්කර් වංශික ද්රවිඩ රජවරු පවා ශ්රී දළදා මාලිගාව වැඩි දියුණු කොට ආරක්ෂා කළහ දළදා මාලිගාව පිහිටා ඇති මහනුවර නගරයම යුනෙස්කෝව විසින් ලෝක උරුමයක් ලෙස නම් කර ඇත',
 'question_translated': 'දැනට දක්නට ලැබෙන පැරණි දෙමහල් වැඩසිටින මාලිගාව ඉදිකරනු ලැබ ඇත්තේ කවුරුන් විසින්ද',
 'text_translated': 'ශ්රී වීරපරාක්රම නරේන්ද්රසිංහ'}

In [ ]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question_translated'], example['context_translated'])
    example['target_text'] = '%s </s>' % example['text_translated']
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=1024)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=32)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }
    

    return encodings

In [ ]:
eval = eval.map(add_eos_to_examples, load_from_cache_file=False)
eval = eval.map(convert_to_features, batched=True, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
eval.set_format(type='torch', columns=columns)

  0%|          | 0/517 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: T

In [ ]:
eval

Dataset({
    features: ['answer_end', 'answer_start', 'attention_mask', 'context_translated', 'input_ids', 'input_text', 'question_translated', 'target_attention_mask', 'target_ids', 'target_text', 'text_translated'],
    num_rows: 517
})

In [ ]:
len(eval)

517

In [ ]:
# cach the dataset, so we can load it directly for training

torch.save(eval, 'eval_data.pt')

In [ ]:
eval[0]['input_ids']

tensor([ 7680,   267, 25454,  ...,     0,     0,     0])

In [ ]:
eval_dataset = torch.load('eval_data.pt')
dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=1)

In [ ]:
answers = []
for batch in tqdm(dataloader):
  outs = model.generate(input_ids=batch['input_ids'], 
                        attention_mask=batch['attention_mask'],
                        max_length=32,
                        early_stopping=True)
  outs = [tokenizer.decode(ids) for ids in outs]
  answers.extend(outs)

  0%|          | 0/517 [00:00<?, ?it/s]

In [ ]:
predictions = []
references = []

for ref, pred in zip(eval_dataset['text_translated'], answers):
  pred = pred.replace("<pad>", "").replace("</s>", "").rstrip().lstrip()
  predictions.append(pred)
  references.append(ref)

In [ ]:
predictions[100], references[100]

('වෝටර්ලූ', 'වෝටර්ලූ ')

In [ ]:
evaluate(references, predictions)

{'exact_match': 27.46615087040619, 'f1': 69.37162590403297}